In [1]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json

c:\Users\moyin\Documents\coding\projects_next\ai-rate-my-professor\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create a Pinecone index
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [3]:
# Load the review data
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. John Smith',
  'subject': 'Introduction to Robotics',
  'stars': 5,
  'review': 'Excellent professor with deep knowledge. Engaging lectures.'},
 {'professor': 'Dr. Emily Johnson',
  'subject': 'Advanced Calculus',
  'stars': 4,
  'review': 'Very knowledgeable, but lectures can be a bit fast-paced.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Physics 101',
  'stars': 3,
  'review': "The course content is good, but the professor's teaching style is monotonous."},
 {'professor': 'Dr. Sarah Wilson',
  'subject': 'Machine Learning',
  'stars': 5,
  'review': 'Fantastic course! Dr. Wilson explains complex topics clearly.'},
 {'professor': 'Dr. David Lee',
  'subject': 'Organic Chemistry',
  'stars': 2,
  'review': 'Difficult to understand. The lectures lack structure.'},
 {'professor': 'Dr. Jennifer Davis',
  'subject': 'Data Structures',
  'stars': 4,
  'review': 'Great content, but more practical examples would be helpful.'},
 {'professor': 'Dr. James Martinez',

In [4]:
processed_data = []
client = OpenAI()

# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )

In [6]:
# Insert the embeddings into the Pinecone index
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

Upserted count: 20


In [8]:
# Print index statistics
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}
